In [8]:
from transformers import GPT2Config, GPT2LMHeadModel
import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data.dataloader import DataLoader
from transformers import BertTokenizer, get_linear_schedule_with_warmup
from tqdm import tqdm
from datetime import datetime
import os
import numpy as np
import logging
import random
from torch.utils.tensorboard import SummaryWriter

In [9]:
def seed_everything(seed: int = 42):
    """Util to make training reproducible"""
    random.seed(seed)

    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)

    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    if os.getenv("CUBLAS_WORKSPACE_CONFIG") is not None:
        torch.use_deterministic_algorithms(True)
        os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def worker_init(worked_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

def set_logger(path):

    logger = logging.getLogger()
    handler = logging.FileHandler(path + "/train_log.txt")
    logger.setLevel(level=logging.INFO)
    handler.setLevel(logging.INFO)
    formatter = logging.Formatter(
        "%(asctime)s - %(filename)s - %(funcName)s - %(lineno)s - %(levelname)s\n%(message)s",
        "%Y-%m-%d %H:%M:%S",
    )
    handler.setFormatter(formatter)
    console = logging.StreamHandler()
    console.setFormatter(formatter)
    logger.addHandler(handler)
    logger.addHandler(console)


class Chinese_Medical_DS(Dataset):
    def __init__(self, path, tokenizer, max_len=1024):
        self.path = path
        sentence = []
        with open(self.path, 'r', encoding='utf-8') as f:
            for line in f:
                sen_ids = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(line.strip()))
                full_sen = []
                full_sen.append(tokenizer.convert_tokens_to_ids('[MASK]'))
                full_sen.extend(sen_ids)
                full_sen.append(tokenizer.convert_tokens_to_ids('[CLS]'))
                if len(full_sen) <= max_len:
                    sentence.append(full_sen)
        self.data = sentence
        
    # need to overload
    def __len__(self):
        return len(self.data)

    # need to overload
    def __getitem__(self, idx):
        input = self.data[idx]
        target = input
        return input, target
    
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""

    def __init__(self, save_path, patience=2, verbose=True, delta=0):
        """
        Args:
            save_path : 模型保存文件夹
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement.
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
        """
        self.save_path = save_path
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        """Saves model when validation loss decrease."""
        if self.verbose:
            print(
                f"Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ..."
            )
        model_to_save = model.module if hasattr(model, 'module') else model
        model_to_save.save_pretrained(self.save_path + 'best_model')
        
        # path = os.path.join(self.save_path, "best_network.pth")
        # torch.save(model.state_dict(), path)  # 这里会存储迄今最优模型的参数
        self.val_loss_min = val_loss

In [10]:
seed_everything()
tok_path = '..\\Raw_GPT2\\vocab.txt'
pretrain_model_path = "..\\Raw_GPT2\\"
output_dir = "model\\"

epochs = 50
warmup_steps = 1000
lr = 1e-5
gradient_accumulation = 18
max_grad_norm = 1.0
log_step = 10000
set_logger(output_dir)
logger = logging.getLogger(__name__)


In [11]:
tokenizer = BertTokenizer(vocab_file=tok_path)
model = GPT2LMHeadModel.from_pretrained(pretrain_model_path)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
logger.info('using device:{}'.format(device))
model.train()
model.to(device)
logger.info(model)

2023-03-25 21:21:13 - 945538542.py - <module> - 4 - INFO
using device:cuda
2023-03-25 21:21:13 - 945538542.py - <module> - 4 - INFO
using device:cuda
2023-03-25 21:21:13 - 945538542.py - <module> - 7 - INFO
GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(21128, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=

In [12]:
train_dataset = Chinese_Medical_DS("..\\Data\\tiny_train.txt", tokenizer)
train_dataloader = DataLoader(dataset=train_dataset, worker_init_fn=worker_init)
valid_dataset = Chinese_Medical_DS("..\\Data\\tiny_valid.txt", tokenizer)
valid_dataloader = DataLoader(dataset=valid_dataset, worker_init_fn=worker_init)
logger.info("len(train_dataloader), len(valid_dataloader) = {}, {}".format(len(train_dataloader), len(valid_dataloader)))

2023-03-25 21:21:28 - 854085413.py - <module> - 5 - INFO
len(train_dataloader), len(valid_dataloader) = 9000, 1500
2023-03-25 21:21:28 - 854085413.py - <module> - 5 - INFO
len(train_dataloader), len(valid_dataloader) = 9000, 1500


In [13]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps,
                                                          num_training_steps=len(train_dataloader))
tb_path = output_dir + "/tb"
if not os.path.exists(tb_path):
    os.mkdir(tb_path)
writer = SummaryWriter(tb_path)

In [14]:
running_loss = 0
early_stopping = EarlyStopping(output_dir)
train_step_per_epoch = len(train_dataloader)
valid_step_per_epoch = len(valid_dataloader)
for epoch in range(epochs):
    logger.info('epoch {}'.format(epoch + 1))
    now = datetime.now()
    logger.info('time: {}'.format(now))
    model.train()
    train_pbar = tqdm(train_dataloader)
    all_train_loss = 0.0
    train_pbar.set_description('epoch-' + str(epoch + 1))
    for step, (input, label) in enumerate(train_pbar):
        input_ids = torch.tensor(label).long().to(device)
        label_ids = torch.tensor(input).long().to(device)

        #  forward pass
        outputs = model(input_ids=input_ids, labels=label_ids)
        loss, logits = outputs[:2]
        
        if gradient_accumulation > 1:
            loss = loss / gradient_accumulation
            
        #  loss backward
        # if fp16:
        #     with amp.scale_loss(loss, optimizer) as scaled_loss:
        #         scaled_loss.backward()
        #         torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), max_grad_norm)
        # else:
        #     loss.backward()
        #     torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        
        loss.backward()
        loss = loss.detach()
        all_train_loss += loss
        
        writer.add_scalar('loss/train_step_loss', scalar_value=loss * gradient_accumulation, global_step=epoch * train_step_per_epoch+step)
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        #  optimizer step
        if (step + 1) % gradient_accumulation == 0:
            running_loss += loss.item()
            optimizer.step()
            optimizer.zero_grad()
            scheduler.step()
        if (step + 1) % log_step == 0:
            logger.info('now time: {}:{}. Step {} of epoch {}, loss {}'.format(
                datetime.now().hour,
                datetime.now().minute,
                (step + 1) // gradient_accumulation,
                epoch + 1,
                running_loss / log_step))
            running_loss = 0
        
        train_pbar.set_postfix({'loss': '{:.7f}'.format(loss*gradient_accumulation)})
        
    logger.info('train step = {}'.format(step))
    all_train_loss = all_train_loss / (step + 1)

    writer.add_scalar('loss/train_epoch_loss', scalar_value=all_train_loss * gradient_accumulation, global_step=epoch + 1)
    logger.info('saving model for epoch {}'.format(epoch + 1))
    if not os.path.exists(output_dir + 'model_epoch{}'.format(epoch + 1)):
        os.mkdir(output_dir + 'model_epoch{}'.format(epoch + 1))
    model_to_save = model.module if hasattr(model, 'module') else model
    model_to_save.save_pretrained(output_dir + 'model_epoch{}'.format(epoch + 1))

    logger.info('epoch {} finished, train loss = {:.10f}'.format(epoch + 1, all_train_loss * gradient_accumulation))

    then = datetime.now()
    logger.info('time: {}'.format(then))
    logger.info('time for one epoch: {}'.format(then - now))
    
    logger.info('start validate')
    model.eval()
    all_valid_loss = 0.0
    valid_pbar = tqdm(valid_dataloader)
    valid_pbar.set_description('valid ' + str(epoch + 1))
    for step, (input, label) in enumerate(valid_pbar):
        input_ids = torch.tensor(label).long().to(device)
        label_ids = torch.tensor(input).long().to(device)

        #  forward pass
        outputs = model(input_ids=input_ids, labels=label_ids)
        loss = outputs[0].detach()
        writer.add_scalar('loss/valid_step_loss', scalar_value=loss, global_step=epoch * valid_step_per_epoch + step)
        all_valid_loss += loss
        valid_pbar.set_postfix({'loss': '{:.7f}'.format(loss)})
    
    logger.info('valid step = {}'.format(step))
    all_valid_loss = all_valid_loss / (step + 1)
    writer.add_scalar('loss/valid_epoch_loss', scalar_value=all_valid_loss, global_step=epoch+1)
    logger.info('valid finished, valid loss = {:.10f}'.format(all_valid_loss))
    early_stopping(all_valid_loss, model)
    if early_stopping.early_stop:
        logger.info("Early stopping")
        break

writer.close()    

logger.info('training finished')
if not os.path.exists(output_dir + 'final_model'):
    os.mkdir(output_dir + 'final_model')
model_to_save = model.module if hasattr(model, 'module') else model
model_to_save.save_pretrained(output_dir + 'final_model')

2023-03-25 21:21:28 - 3756389727.py - <module> - 6 - INFO
epoch 1
2023-03-25 21:21:28 - 3756389727.py - <module> - 6 - INFO
epoch 1
2023-03-25 21:21:28 - 3756389727.py - <module> - 8 - INFO
time: 2023-03-25 21:21:28.698780
2023-03-25 21:21:28 - 3756389727.py - <module> - 8 - INFO
time: 2023-03-25 21:21:28.698780
epoch-1: 100%|██████████| 9000/9000 [08:35<00:00, 17.45it/s, loss=2.8717773]
2023-03-25 21:30:04 - 3756389727.py - <module> - 58 - INFO
train step = 8999
2023-03-25 21:30:04 - 3756389727.py - <module> - 58 - INFO
train step = 8999
2023-03-25 21:30:04 - 3756389727.py - <module> - 62 - INFO
saving model for epoch 1
2023-03-25 21:30:04 - 3756389727.py - <module> - 62 - INFO
saving model for epoch 1
2023-03-25 21:30:04 - 3756389727.py - <module> - 68 - INFO
epoch 1 finished, train loss = 2.6329622269
2023-03-25 21:30:04 - 3756389727.py - <module> - 68 - INFO
epoch 1 finished, train loss = 2.6329622269
2023-03-25 21:30:04 - 3756389727.py - <module> - 71 - INFO
time: 2023-03-25 21:30

Validation loss decreased (inf --> 2.404044).  Saving model ...


2023-03-25 21:30:35 - 3756389727.py - <module> - 6 - INFO
epoch 2
2023-03-25 21:30:35 - 3756389727.py - <module> - 6 - INFO
epoch 2
2023-03-25 21:30:35 - 3756389727.py - <module> - 8 - INFO
time: 2023-03-25 21:30:35.281586
2023-03-25 21:30:35 - 3756389727.py - <module> - 8 - INFO
time: 2023-03-25 21:30:35.281586
epoch-2: 100%|██████████| 9000/9000 [08:33<00:00, 17.51it/s, loss=2.6565728]
2023-03-25 21:39:09 - 3756389727.py - <module> - 58 - INFO
train step = 8999
2023-03-25 21:39:09 - 3756389727.py - <module> - 58 - INFO
train step = 8999
2023-03-25 21:39:09 - 3756389727.py - <module> - 62 - INFO
saving model for epoch 2
2023-03-25 21:39:09 - 3756389727.py - <module> - 62 - INFO
saving model for epoch 2
2023-03-25 21:39:09 - 3756389727.py - <module> - 68 - INFO
epoch 2 finished, train loss = 2.3718743324
2023-03-25 21:39:09 - 3756389727.py - <module> - 68 - INFO
epoch 2 finished, train loss = 2.3718743324
2023-03-25 21:39:09 - 3756389727.py - <module> - 71 - INFO
time: 2023-03-25 21:39

Validation loss decreased (2.404044 --> 2.243327).  Saving model ...


2023-03-25 21:39:38 - 3756389727.py - <module> - 6 - INFO
epoch 3
2023-03-25 21:39:38 - 3756389727.py - <module> - 6 - INFO
epoch 3
2023-03-25 21:39:38 - 3756389727.py - <module> - 8 - INFO
time: 2023-03-25 21:39:38.960347
2023-03-25 21:39:38 - 3756389727.py - <module> - 8 - INFO
time: 2023-03-25 21:39:38.960347
epoch-3: 100%|██████████| 9000/9000 [08:36<00:00, 17.42it/s, loss=2.5512846]
2023-03-25 21:48:15 - 3756389727.py - <module> - 58 - INFO
train step = 8999
2023-03-25 21:48:15 - 3756389727.py - <module> - 58 - INFO
train step = 8999
2023-03-25 21:48:15 - 3756389727.py - <module> - 62 - INFO
saving model for epoch 3
2023-03-25 21:48:15 - 3756389727.py - <module> - 62 - INFO
saving model for epoch 3
2023-03-25 21:48:15 - 3756389727.py - <module> - 68 - INFO
epoch 3 finished, train loss = 2.2253904343
2023-03-25 21:48:15 - 3756389727.py - <module> - 68 - INFO
epoch 3 finished, train loss = 2.2253904343
2023-03-25 21:48:15 - 3756389727.py - <module> - 71 - INFO
time: 2023-03-25 21:48

...

2023-03-26 04:27:34 - 3756389727.py - <module> - 6 - INFO
epoch 49
2023-03-26 04:27:34 - 3756389727.py - <module> - 6 - INFO
epoch 49
2023-03-26 04:27:34 - 3756389727.py - <module> - 8 - INFO
time: 2023-03-26 04:27:34.762547
2023-03-26 04:27:34 - 3756389727.py - <module> - 8 - INFO
time: 2023-03-26 04:27:34.762547
epoch-49: 100%|██████████| 9000/9000 [08:22<00:00, 17.91it/s, loss=2.3344057]
2023-03-26 04:35:57 - 3756389727.py - <module> - 58 - INFO
train step = 8999
2023-03-26 04:35:57 - 3756389727.py - <module> - 58 - INFO
train step = 8999
2023-03-26 04:35:57 - 3756389727.py - <module> - 62 - INFO
saving model for epoch 49
2023-03-26 04:35:57 - 3756389727.py - <module> - 62 - INFO
saving model for epoch 49
2023-03-26 04:35:57 - 3756389727.py - <module> - 68 - INFO
epoch 49 finished, train loss = 1.8478950262
2023-03-26 04:35:57 - 3756389727.py - <module> - 68 - INFO
epoch 49 finished, train loss = 1.8478950262
2023-03-26 04:35:57 - 3756389727.py - <module> - 71 - INFO
time: 2023-03-2

Validation loss decreased (1.958709 --> 1.958709).  Saving model ...


2023-03-26 04:36:27 - 3756389727.py - <module> - 6 - INFO
epoch 50
2023-03-26 04:36:27 - 3756389727.py - <module> - 6 - INFO
epoch 50
2023-03-26 04:36:27 - 3756389727.py - <module> - 8 - INFO
time: 2023-03-26 04:36:27.378762
2023-03-26 04:36:27 - 3756389727.py - <module> - 8 - INFO
time: 2023-03-26 04:36:27.378762
epoch-50: 100%|██████████| 9000/9000 [08:23<00:00, 17.89it/s, loss=2.2824733]
2023-03-26 04:44:50 - 3756389727.py - <module> - 58 - INFO
train step = 8999
2023-03-26 04:44:50 - 3756389727.py - <module> - 58 - INFO
train step = 8999
2023-03-26 04:44:50 - 3756389727.py - <module> - 62 - INFO
saving model for epoch 50
2023-03-26 04:44:50 - 3756389727.py - <module> - 62 - INFO
saving model for epoch 50
2023-03-26 04:44:50 - 3756389727.py - <module> - 68 - INFO
epoch 50 finished, train loss = 1.8477205038
2023-03-26 04:44:50 - 3756389727.py - <module> - 68 - INFO
epoch 50 finished, train loss = 1.8477205038
2023-03-26 04:44:50 - 3756389727.py - <module> - 71 - INFO
time: 2023-03-2

Validation loss decreased (1.958709 --> 1.958709).  Saving model ...


2023-03-26 04:45:20 - 3756389727.py - <module> - 101 - INFO
training finished
2023-03-26 04:45:20 - 3756389727.py - <module> - 101 - INFO
training finished
